In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

from pathlib import Path

import logging
logging.basicConfig(level=logging.INFO)

import numpy

In [ ]:
from structlib import AtomCell, SimpleAtoms
from structlib.visualize import show_atoms_3d
from structlib.disloc import disloc_loop_z

In [ ]:
cell: AtomCell = AtomCell.read(Path('AlN-ortho.xsf'))

print(str(cell.cell_size))

cell = cell.repeat((8, 5, 5), explode=True)

show_atoms_3d(cell);

In [ ]:
import polars

frame = cell.get_atoms('local')

frame.select((polars.col('x') > 0.))

In [ ]:
loop = disloc_loop_z(cell, center=[12., 12., 10.], b=[0., 0., -5.02/2.,], loop_r=10.)

#show_atoms_3d(loop);
show_atoms_3d(loop.crop_atoms(x_min=5., x_max=15.));
#loop